In [ ]:
import os
import sys
import re
import pandas as pd
from functools import partial
sys.path.append("/home/kai/muku_esg/ESG-BERT/")
from allgrammer import grammer
from pdfminer.high_level import extract_text
%cd /home/kai/muku_esg/ESG-BERT/

In [ ]:
def ascii(text):
    text = re.sub(r'[^\x00-\x7F]', '', text)
    text = re.sub(r'[\x0C]', r"\n\n", text)
    return text

In [ ]:
# folder = 'taiwan_pdf'
# csv_folder = 'predict_taiwan'
# word_cnt = 'eight_new'

In [ ]:
folder = 'american_pdf'
csv_folder = 'predict_american'
word_cnt = 'eight_new'

In [ ]:
def preprocess(x, folder, file_name):
    x = x.replace('\n', ' ')
    x = re.sub(r'[!\"#$%&\'()*\+,-.\/:;<=>?@\[\\\]^_`{|}~]', '', x)
    x = x.replace('\t', ' ')
    x = x.strip()
    if len(x.split()) < 8:
        return None
    else:
        return grammer(x, folder, file_name)

In [ ]:
file_name = '06_bxp_1_65'
text = extract_text(f"./{folder}/{file_name}.pdf")
text = ascii(text)
text = text.strip('\"')
paragraph = re.split(r'\n\s*\n', text)
constant_preprocess = partial(preprocess, folder=folder, file_name=file_name)
paragraph = map(constant_preprocess, paragraph)
paragraph = list(paragraph)
# for i in range(len(paragraph)):
#     print(paragraph[i])
y = []
for i in range(len(paragraph)):
    if paragraph[i] != None:
        x = paragraph[i].split()
        fifthlist = x[:300]
        fifthstr = ' '.join(fifthlist)
        y.append(fifthstr)
paragraph = y
for i in range(len(paragraph)):
    print(paragraph[i])
# paragraph = [x for x in paragraph if x is not None]
# paragraph = [x[:500] for x in paragraph if x is not None]

In [ ]:
%cd predict_american
with open("./output.txt", "w") as file:
    file.truncate()
with open("./paragraph.txt", "w") as file:
    file.truncate()
for i in range(len(paragraph)):
    # Open a file in write mode
    with open("./paragraph.txt", "w") as f:
        # Write the strings to the file
        f.writelines(paragraph[i])
    # os.system(f'curl -X POST http://127.0.0.1:8080/predictions/bert -T paragraph.txt >> output.txt');
    os.system(f'curl -X POST http://127.0.0.1:8090/predictions/bert -T paragraph.txt >> output.txt')
    os.system(f'echo >> output.txt')

In [ ]:
# labels = pd.read_csv('./predict_file/output.txt', names=['labels'])
with open('./output.txt', 'r') as f:
    labels = f.readlines()
    labels = [x.strip('\n') for x in labels]
ans = pd.DataFrame({'paragraph': paragraph, 'predict_label': labels})
ans.to_csv(f'../{csv_folder}/{word_cnt}/{file_name}.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
file_names = ['15_msi_1_97.pdf',
   '16_gigabyte_1_74.pdf', '17_nanya_1_154.pdf', '18_novatek_1_140.pdf', '19_mediatek_1_97.pdf', '20_asia_1_119.pdf', 
   '21_cathay_1_52.pdf', '22_cht_1_79.pdf', '23_fubon_1_89.pdf', '24_mega_1_68.pdf']
for file_name in file_names:
    file_name = file_name.strip('.pdf')
    os.chdir('/home/kai/muku_esg/ESG-BERT')
    text = extract_text(f"./{folder}/{file_name}.pdf")
    text = ascii(text)
    text = text.strip('\"')
    paragraph = re.split(r'\n\s*\n', text)
    constant_preprocess = partial(preprocess, folder=folder, file_name=file_name)
    paragraph = map(constant_preprocess, paragraph)
    paragraph = list(paragraph)
    y = []
    for i in range(len(paragraph)):
        if paragraph[i] != None:
            x = paragraph[i].split()
            fifthlist = x[:300]
            fifthstr = ' '.join(fifthlist)
            y.append(fifthstr)
    paragraph = y
    #   predict
    os.chdir('./predict_american')
    with open("./output.txt", "w") as file:
        file.truncate()
    with open("./paragraph.txt", "w") as file:
        file.truncate()
    for i in range(len(paragraph)):
        # Open a file in write mode
        with open("./paragraph.txt", "w") as f:
            # Write the strings to the file
            f.writelines(paragraph[i])
        # os.system(f'curl -X POST http://127.0.0.1:8080/predictions/bert -T paragraph.txt >> output.txt');
        os.system(f'curl -X POST http://127.0.0.1:8080/predictions/bert -T paragraph.txt >> output.txt')
        os.system(f'echo >> output.txt')
    #   read output label and create csv
    with open('./output.txt', 'r') as f:
        labels = f.readlines()
        labels = [x.strip('\n') for x in labels]
    ans = pd.DataFrame({'paragraph': paragraph, 'predict_label': labels})
    ans.to_csv(f'../{csv_folder}/{word_cnt}/{file_name}.csv', sep=',', index=False, encoding='utf-8')

In [ ]:
file_list = os.listdir('./taiwan_pdf/')

In [ ]:
taiwan = [
'01_umc_1_154.pdf', '02_tsmc_1_210.pdf', '03_macronix_1_114.pdf', '04_esun_1_154.pdf', '05_eme_1_46.pdf',
 '06_asus_1_108.pdf', '07_acer_1_127.pdf', '08_witron_1_93.pdf', '09_honhai_1_22.pdf', '10_compal_1_169.pdf',
  '11_quanta_1_123.pdf', '12_formosa_1_18.pdf', '13_csc_1_150.pdf', '14_qisda_1_134.pdf', '15_msi_1_97.pdf',
   '16_gigabyte_1_74.pdf', '17_nanya_1_154.pdf', '18_novatek_1_140.pdf', '19_mediatek_1_97.pdf', '20_asia_1_119.pdf', 
   '21_cathay_1_52.pdf', '22_cht_1_79.pdf', '23_fubon_1_89.pdf', '24_mega_1_68.pdf']

In [ ]:
file_list = [
'01_dow_1_85.pdf', '02_agilent_1_106.pdf', '03_amazon_1_79.pdf', '04_apple_1_72.pdf',
'05_boeing_1_66.pdf', '06_bxp_1_65.pdf', '07_charles_1_50.pdf', '08_cisco_1_56.pdf', '09_citigroup_1_137.pdf', '10_cme_1_34.pdf', 
'11_colgate_1_84.pdf', '12_corning_1_71.pdf', '13_expeditor_1_37.pdf', '14_eei_1_80.pdf', '15_itt_1_44.pdf', 
'16_fedex_1_34.pdf', '17_firstscolar_1_57.pdf', '18_google_1_14.pdf', '19_intel_1_86.pdf', '20_jpmorgan_1_61.pdf', 
'21_microsoft_1_89.pdf', '22_rockwell_1_58.pdf', '23_ibm_1_49.pdf', '24_traveler_1_147.pdf', '25_visa_1_52.pdf']

In [ ]:
file_list = sorted(file_list)

In [ ]:
print(file_list)

In [ ]:
import csv
# Open the CSV file
os.chdir('/home/kai/muku_esg/ESG-BERT/predict_taiwan/five_word')
file_name = '08_witron_1_93'
with open(f'./{file_name}.csv', 'r') as f:
    # Create a CSV reader object
    reader = csv.reader(f)
    paragraphs = []
    labels = []
    for row in reader:
        if row[0][:27] == '2021 Wistron ITS ESG Report' :
            pass
        else:
            paragraphs.append(row[0])
            labels.append(row[1])
    ans = pd.DataFrame({'paragraph': paragraphs, 'predict_label': labels})
    ans.to_csv(f'../five_new/{file_name}.csv', sep=',', index=False, encoding='utf-8')